## Download repo

In [2]:
import os
import shutil
import zipfile
import urllib.request

def download_repo(url, save_to):
    zip_filename = save_to + '.zip'
    urllib.request.urlretrieve(url, zip_filename)
    
    if os.path.exists(save_to):
        shutil.rmtree(save_to)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    del zip_ref
    assert os.path.exists(save_to)

In [3]:
REPO_PATH = 'LinearizedNNs-master'

download_repo(url='https://github.com/maxkvant/LinearizedNNs/archive/master.zip', save_to=REPO_PATH)

## Imports

In [4]:
import sys
sys.path.append(f"{REPO_PATH}/src")

In [5]:
import time
import numpy as np
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.datasets import FashionMNIST

from estimator import Estimator
from pytorch_impl.estimators import SgdEstimator
from pytorch_impl.nns import Myrtle5, Myrtle7, Myrtle10
from pytorch_impl import ClassifierTraining
from pytorch_impl.matrix_exp import matrix_exp, compute_exp_term
from pytorch_impl.nns.utils import to_one_hot, print_sizes
from from_neural_kernels import to_zca, CustomTensorDataset, get_cifar_zca

In [6]:
device = torch.device('cuda:0') if (torch.cuda.is_available()) else torch.device('cpu')
device

device(type='cuda', index=0)

## Cifar 10

In [65]:
class GpEstimator(Estimator):
    def __init__(self, models, n_classes, learning_rate, x_example, device, groups=1):
        super(GpEstimator, self).__init__()
        self.models    = [model.to(device) for model in models]
        self.lr        = learning_rate
        self.n_classes = n_classes
        self.device    = device
        
        n = len(models)
        X = torch.stack([x_example]).to(device)
        
        model = models[0].to(device)
        readout_size = model.readout(X).size()[1]
    
        # TODO: Assert that models have the same readout size
        
        self.w      = torch.zeros([n, readout_size, n_classes]).to(device)
        self.w_size = n * groups
        
    def get_w_update(self, X, right_vector):
        with torch.no_grad():
            assert len(X) == len(right_vector)

            X            = X.to(self.device)
            right_vector = right_vector.to(self.device)

            n = len(X)
            w_updates = []
            
            for model in self.models:
                features = self.to_model_features(X, model)
                update = torch.matmul(features.T, right_vector)
                w_updates.append(update)
            return torch.stack(w_updates)
                                  
    def to_model_features(self, X, model):
        with torch.no_grad():
            model = model.to(device)
            return model.readout(X) * (1. / np.sqrt(self.w_size))
        
    def calc_kernel(self, X):
        with torch.no_grad():
            X = X.to(device)
            
            res = torch.zeros([len(X), len(X)]).to(device).double()
            for model in self.models:
                features_x = self.to_model_features(X, model)
                
                res += torch.matmul(features_x, features_x.T).double()
            return res.float()
        
    def calc_kernel_pred(self, X):
        with torch.no_grad():
            X = X.to(device)
            
            n = len(X)
            y_pred = torch.zeros([n, self.n_classes]).to(device).double()
            kernel = torch.zeros([len(X), len(X)]).to(device).double()
            
            for model, w in zip(self.models, self.w):
                features = self.to_model_features(X, model)
                
                kernel += torch.matmul(features, features.T).double()
                y_pred += torch.matmul(features, w).double()
            return kernel, y_pred.float()
        
    def calc_kernels(self, X_train, X_test):
        with torch.no_grad():
            X_train = X_train.to(device)
            X_test  = X_test.to(device)
            
            res_train = torch.zeros([len(X_train), len(X_train)]).to(device)
            res_test  = torch.zeros([len(X_test),  len(X_train)]).to(device)
            for model in self.models:
                features_train = self.to_model_features(X_train, model)
                features_test  = self.to_model_features(X_test,  model)
                
                res_train += torch.matmul(features_train, features_train.T)
                res_test  += torch.matmul(features_test,  features_train.T)
            return res_train, res_test
            
    def predict(self, X, cur_w=None):
        X = X.to(self.device)
        if cur_w is None:
            cur_w = self.w
         
        with torch.no_grad():
            n = len(X)
            res = torch.zeros([n, self.n_classes]).double().to(device)

            for model, w in zip(self.models, cur_w):
                features = self.to_model_features(X, model)
                res += torch.matmul(features, w).double()
            return res.float()

In [8]:
def calc_right_vector(kernel, y, learning_rate=1e5, reg_param=0):
    with torch.no_grad():
        y      = y.to(device)
        kernel = kernel.to(device)
        
        n      = len(kernel)
        reg = torch.eye(n).to(device) * reg_param
        
        exp_term = - learning_rate * compute_exp_term(- learning_rate * (kernel + reg), device)
        right_vector = torch.matmul(exp_term, - y)
        return right_vector

In [42]:
%%time

X_train, labels_train, X_test, labels_test = get_cifar_zca()

N_train = 12800
N_test  = 1000

X_train      = torch.tensor(X_train[:N_train]).float()
labels_train = torch.tensor(labels_train[:N_train], dtype=torch.long)

X_test       = torch.tensor(X_test[:N_test]).float()
labels_test  = torch.tensor(labels_test[:N_test],  dtype=torch.long)

num_classes = 10

y_train = to_one_hot(labels_train, num_classes).to(device)
y_test  = to_one_hot(labels_test,  num_classes).to(device)

Files already downloaded and verified
Files already downloaded and verified
CPU times: user 3min 31s, sys: 1min, total: 4min 31s
Wall time: 40.6 s


In [43]:
# y_train = np.load('../data/cifar10_targets_2.npz')['targets'][:N_train]

In [52]:
cifar_train = CustomTensorDataset(torch.tensor(X_train), torch.tensor(y_train).float(), transform='flips')
cifar_test  = CustomTensorDataset(torch.tensor(X_test),  torch.tensor(labels_test, dtype=torch.long))

train_loader = torch.utils.data.DataLoader(cifar_train, batch_size=1280 * 2)
test_loader  = torch.utils.data.DataLoader(cifar_test, batch_size=1000)

/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


## Kernel method

In [62]:
def boosting(estimator, train_loader, test_loader, learning_rate=1e5, n_iter=10):
    output_kernel = False
    
    with torch.no_grad():
        batches_num = 0
        for _ in enumerate(train_loader):
            batches_num += 1
        n_batches = (batches_num * 2) // 3 + 1
        
        test_size = 0
        for _, (X, _) in enumerate(test_loader):
            test_size += len(X)
            
        for iter_num  in range(n_iter):
            print(f"iter {iter_num} ==========================")
            
            betas = []
            w_update = 0
            iter_start = time.time()
            
            for batch_i, (X, y) in enumerate(train_loader):
                if batch_i >= n_batches:
                    break
                
                X = X.to(device)
                y = y.to(device)
                
                kernel, y_pred = estimator.calc_kernel_pred(X)
                if output_kernel:
                    print(f"kernel\n{kernel[:5,:5]}")
                
                y_residual = y_pred - y
                
                train_acc = (y_pred.argmax(dim=1) == y.argmax(dim=1)).float().mean().item()
                train_mse = (y_residual ** 2).mean().item()
                print(f"batch {batch_i}: train_acc {train_acc:.4f}, train_mse {train_mse:.6f}")
                
                right_vector = calc_right_vector(kernel, y_residual, learning_rate=learning_rate)
                
                w_update += estimator.get_w_update(X, right_vector).double()
                
                pred_change = torch.matmul(kernel, right_vector)
                cur_beta    = (- y_residual * pred_change).sum() / (pred_change ** 2).sum()
                
                w_update += cur_w_update
                betas.append(cur_beta.item())
            
            w_update = (w_update / n_batches).float()
            
            _, (X, y) = next(enumerate(train_loader))
            
            y_pred      = estimator.predict(X)
            pred_change = estimator.predict(X, w_update)
            
            y_residual = y_pred - y
            beta    = (- y_residual * pred_change).sum() / (pred_change ** 2).sum().item()
                
            estimator.w += w_update * beta
            
            test_acc = 0
            for _, (X_test, labels) in enumerate(test_loader):
                y_pred = estimator.predict(X_test) 
                test_acc += (y_pred.argmax(dim=1) == labels.to(device)).float().sum().item() / test_size
                
            print(f"iter {iter_num} done. took {time.time() - iter_start:.0f}s. beta {beta:.3f}, test_acc {test_acc:.4f}")
            print()

In [53]:
n_models = 500

# 500 * 50 * 32  = 800k

models = [Myrtle7(num_filters=1, groups=50) for _ in range(n_models)]
n_models

estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device, groups=50)

In [54]:
boosting(estimator, train_loader, test_loader, learning_rate=1e6, n_iter=2)

iter 0 ==========================
batch 0: train_acc 0.1008, train_mse 1.000000
batch 1: train_acc 0.0945, train_mse 1.000000
batch 2: train_acc 0.1059, train_mse 1.000000
batch 3: train_acc 0.1016, train_mse 1.000000
iter 0 done. took 294s test_acc 0.7500

iter 1 ==========================
batch 0: train_acc 0.9004, train_mse 0.144447
batch 1: train_acc 0.8973, train_mse 0.145056
batch 2: train_acc 0.8918, train_mse 0.144826
batch 3: train_acc 0.8965, train_mse 0.144527
iter 1 done. took 294s test_acc 0.7680



In [55]:
boosting(estimator, train_loader, test_loader, learning_rate=1e6, n_iter=8)

iter 0 ==========================
batch 0: train_acc 0.9480, train_mse 0.104977
batch 1: train_acc 0.9523, train_mse 0.106574
batch 2: train_acc 0.9508, train_mse 0.106762
batch 3: train_acc 0.9465, train_mse 0.105559
iter 0 done. took 294s test_acc 0.7660

iter 1 ==========================
batch 0: train_acc 0.9809, train_mse 0.082550
batch 1: train_acc 0.9734, train_mse 0.085491
batch 2: train_acc 0.9691, train_mse 0.085250
batch 3: train_acc 0.9742, train_mse 0.084768
iter 1 done. took 294s test_acc 0.7790

iter 2 ==========================
batch 0: train_acc 0.9867, train_mse 0.066001
batch 1: train_acc 0.9848, train_mse 0.069532
batch 2: train_acc 0.9871, train_mse 0.069126
batch 3: train_acc 0.9852, train_mse 0.067711
iter 2 done. took 294s test_acc 0.7830

iter 3 ==========================
batch 0: train_acc 0.9918, train_mse 0.057009
batch 1: train_acc 0.9941, train_mse 0.056957
batch 2: train_acc 0.9930, train_mse 0.056395
batch 3: train_acc 0.9941, train_mse 0.055626
iter 3 d

In [33]:
estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device)
boosting(estimator, train_loader, test_loader, learning_rate=1e4)

iter 0 ==========================
batch 0: train_acc 0.0977, train_mse 7.183765
batch 1: train_acc 0.1037, train_mse 7.173288
batch 2: train_acc 0.0973, train_mse 7.196385
iter 0 done. took 100s test_acc 0.6620

iter 1 ==========================
batch 0: train_acc 0.6648, train_mse 3.893153
batch 1: train_acc 0.6594, train_mse 3.899528
batch 2: train_acc 0.6680, train_mse 3.843274
iter 1 done. took 102s test_acc 0.6920

iter 2 ==========================
batch 0: train_acc 0.7027, train_mse 3.694586
batch 1: train_acc 0.7041, train_mse 3.693863
batch 2: train_acc 0.7086, train_mse 3.654493
iter 2 done. took 99s test_acc 0.6800

iter 3 ==========================
batch 0: train_acc 0.7195, train_mse 3.575631
batch 1: train_acc 0.7086, train_mse 3.580958
batch 2: train_acc 0.7141, train_mse 3.561848
iter 3 done. took 102s test_acc 0.7100

iter 4 ==========================
batch 0: train_acc 0.7357, train_mse 3.459083
batch 1: train_acc 0.7262, train_mse 3.465120
batch 2: train_acc 0.7371, 

In [34]:
estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device)
boosting(estimator, train_loader, test_loader, learning_rate=1e6)

iter 0 ==========================
batch 0: train_acc 0.0977, train_mse 7.183765
batch 1: train_acc 0.1037, train_mse 7.173288
batch 2: train_acc 0.0973, train_mse 7.196385
iter 0 done. took 99s test_acc 0.6670

iter 1 ==========================
batch 0: train_acc 0.6859, train_mse 4.011244
batch 1: train_acc 0.6986, train_mse 3.945497
batch 2: train_acc 0.7074, train_mse 3.875035
iter 1 done. took 103s test_acc 0.6880

iter 2 ==========================
batch 0: train_acc 0.7076, train_mse 3.973394
batch 1: train_acc 0.7105, train_mse 3.957006
batch 2: train_acc 0.7211, train_mse 3.942075
iter 2 done. took 98s test_acc 0.6740

iter 3 ==========================
batch 0: train_acc 0.7281, train_mse 3.969994
batch 1: train_acc 0.7188, train_mse 3.965328
batch 2: train_acc 0.7355, train_mse 3.896935
iter 3 done. took 103s test_acc 0.6790

iter 4 ==========================
batch 0: train_acc 0.7303, train_mse 3.971407
batch 1: train_acc 0.7201, train_mse 3.975674
batch 2: train_acc 0.7227, t

In [ ]:
n_models = 5000

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device)

In [30]:
boosting(estimator, train_loader, test_loader)

iter 0 ==========================
batch 0: train_acc 0.0992, train_mse 1.000000
batch 1: train_acc 0.1023, train_mse 1.000000
batch 2: train_acc 0.0867, train_mse 1.000000
batch 3: train_acc 0.1023, train_mse 1.000000
batch 4: train_acc 0.1063, train_mse 1.000000
batch 5: train_acc 0.1055, train_mse 1.000000
batch 6: train_acc 0.1086, train_mse 1.000000
iter 0 done. test_acc 0.6610
iter 1 ==========================
batch 0: train_acc 0.6898, train_mse 0.228092
batch 1: train_acc 0.6898, train_mse 0.229243
batch 2: train_acc 0.6852, train_mse 0.231006
batch 3: train_acc 0.6664, train_mse 0.232013
batch 4: train_acc 0.6750, train_mse 0.231813
batch 5: train_acc 0.6703, train_mse 0.232756
batch 6: train_acc 0.6609, train_mse 0.234007
iter 1 done. test_acc 0.6930
iter 2 ==========================
batch 0: train_acc 0.7344, train_mse 0.212709
batch 1: train_acc 0.7438, train_mse 0.211246
batch 2: train_acc 0.7203, train_mse 0.213644
batch 3: train_acc 0.7297, train_mse 0.215032
batch 4: tra

In [11]:
cifar_train = CustomTensorDataset(torch.tensor(X_train), torch.tensor(labels_train, dtype=torch.long), transform=None)

/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [12]:
n_models = 100

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device)

In [13]:
boosting(estimator, train_loader, test_loader)

iter 0 ==========================
batch 0: train_acc 0.0992, train_mse 1.000000
batch 1: train_acc 0.1023, train_mse 1.000000
batch 2: train_acc 0.0867, train_mse 1.000000
batch 3: train_acc 0.1023, train_mse 1.000000
batch 4: train_acc 0.1063, train_mse 1.000000
batch 5: train_acc 0.1055, train_mse 1.000000
batch 6: train_acc 0.1086, train_mse 1.000000
iter 0 done. took 630s test_acc 0.6570

iter 1 ==========================
batch 0: train_acc 0.6945, train_mse 0.230021
batch 1: train_acc 0.6820, train_mse 0.229634
batch 2: train_acc 0.6617, train_mse 0.232192
batch 3: train_acc 0.6844, train_mse 0.231658
batch 4: train_acc 0.6617, train_mse 0.233073
batch 5: train_acc 0.6609, train_mse 0.234637
batch 6: train_acc 0.6672, train_mse 0.234089
iter 1 done. took 627s test_acc 0.6890

iter 2 ==========================
batch 0: train_acc 0.7398, train_mse 0.210167
batch 1: train_acc 0.7594, train_mse 0.208906
batch 2: train_acc 0.7469, train_mse 0.209019
batch 3: train_acc 0.7258, train_mse

In [14]:
boosting(estimator, train_loader, test_loader, n_iter=5)

iter 0 ==========================
batch 0: train_acc 0.8664, train_mse 0.160979
batch 1: train_acc 0.8602, train_mse 0.162237
batch 2: train_acc 0.8539, train_mse 0.163230
batch 3: train_acc 0.8453, train_mse 0.167403
batch 4: train_acc 0.8555, train_mse 0.163886
batch 5: train_acc 0.8516, train_mse 0.164142
batch 6: train_acc 0.8484, train_mse 0.163351
iter 0 done. took 628s test_acc 0.7280

iter 1 ==========================
batch 0: train_acc 0.8641, train_mse 0.158279
batch 1: train_acc 0.8680, train_mse 0.157631
batch 2: train_acc 0.8727, train_mse 0.158165
batch 3: train_acc 0.8672, train_mse 0.160398
batch 4: train_acc 0.8695, train_mse 0.161058
batch 5: train_acc 0.8625, train_mse 0.159203
batch 6: train_acc 0.8688, train_mse 0.160781
iter 1 done. took 627s test_acc 0.7360

iter 2 ==========================
batch 0: train_acc 0.8781, train_mse 0.157369
batch 1: train_acc 0.8758, train_mse 0.155880
batch 2: train_acc 0.8727, train_mse 0.156660
batch 3: train_acc 0.8602, train_mse

## Full Cifar

In [57]:
%%time

X_train, labels_train, X_test, labels_test = get_cifar_zca()

X_train      = torch.tensor(X_train).float()
labels_train = torch.tensor(labels_train, dtype=torch.long)

X_test       = torch.tensor(X_test).float()
labels_test  = torch.tensor(labels_test,  dtype=torch.long)

num_classes = 10

y_train = to_one_hot(labels_train, num_classes).to(device)
y_test  = to_one_hot(labels_test,  num_classes).to(device)

Files already downloaded and verified
Files already downloaded and verified
CPU times: user 3min 33s, sys: 1min 2s, total: 4min 35s
Wall time: 41 s


In [58]:
# y_train = np.load('../data/cifar10_targets_2.npz')['targets']

In [59]:
cifar_train = CustomTensorDataset(torch.tensor(X_train), torch.tensor(y_train).float(), transform='flips')
cifar_test  = CustomTensorDataset(torch.tensor(X_test),  torch.tensor(labels_test, dtype=torch.long))

train_loader = torch.utils.data.DataLoader(cifar_train, batch_size=1280 * 2)
test_loader  = torch.utils.data.DataLoader(cifar_test,  batch_size=1280 * 2)

/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [63]:
n_models = 500

# 500 * 50 * 32  = 800k

models = [Myrtle7(num_filters=1, groups=50) for _ in range(n_models)]
n_models

estimator = GpEstimator(models, num_classes, 0.2, X_train[0], device, groups=50)

In [64]:
boosting(estimator, train_loader, test_loader, learning_rate=1e6, n_iter=64)

iter 0 ==========================
batch 0: train_acc 0.1008, train_mse 1.000000
batch 1: train_acc 0.0945, train_mse 1.000000
batch 2: train_acc 0.1059, train_mse 1.000000
batch 3: train_acc 0.1016, train_mse 1.000000
batch 4: train_acc 0.0977, train_mse 1.000000
batch 5: train_acc 0.1086, train_mse 1.000000
batch 6: train_acc 0.0988, train_mse 1.000000
batch 7: train_acc 0.1023, train_mse 1.000000
batch 8: train_acc 0.1000, train_mse 1.000000
batch 9: train_acc 0.1031, train_mse 1.000000
batch 10: train_acc 0.1008, train_mse 1.000000
batch 11: train_acc 0.1012, train_mse 1.000000
batch 12: train_acc 0.0973, train_mse 1.000000
batch 13: train_acc 0.0957, train_mse 1.000000
iter 0 done. took 1115s test_acc 0.7820

iter 1 ==========================
batch 0: train_acc 0.8391, train_mse 0.170746
batch 1: train_acc 0.8371, train_mse 0.172967
batch 2: train_acc 0.8156, train_mse 0.173399
batch 3: train_acc 0.8355, train_mse 0.172503
batch 4: train_acc 0.8215, train_mse 0.173839
batch 5: trai

KeyboardInterrupt: 

In [ ]:
n_models

## Log Reg

In [32]:
n_models = 5000

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

learning_rate = 0.2
estimator = GpEstimator(models, num_classes, learning_rate, X_train[0], device)

n_iter = 20

for iter_ in range(n_iter):
    print(f"iter {iter_} ======================= ")
    
    for batch_i, (X, labels) in enumerate(train_loader):
        batch_start = time.time()
        
        X      = X.to(device)
        labels = labels.to(device)
        
        y_pred = estimator.predict(X)
        train_acc = (y_pred.argmax(dim=1) == labels).float().mean().item()
        print(f"train_acc {train_acc}")
        
        y_pred.requires_grad = True

        loss = nn.CrossEntropyLoss()(y_pred, labels)
        loss.backward()
        grad = y_pred.grad
        
        estimator.w += -learning_rate * estimator.get_w_update(X, grad)
        
        print(f"batch_{batch_i} took {time.time() - batch_start:0}s")
        print()
        
    y_pred = estimator.predict(X_test)
    test_acc = (y_pred.argmax(dim=1) == labels_test.to(device)).float().mean().item()
    print(f"iter {iter_} done, test_acc = {test_acc}")

iter 0 ======================= 
train_acc 0.09921874850988388
batch_0 took 57.80506658554077s

train_acc 0.09140624850988388
batch_1 took 55.33869671821594s

train_acc 0.1328125
batch_2 took 55.32820272445679s

train_acc 0.09296875447034836
batch_3 took 55.33759117126465s

train_acc 0.09375
batch_4 took 55.329872846603394s

train_acc 0.1171875
batch_5 took 55.33920168876648s

train_acc 0.09765625
batch_6 took 55.32954454421997s

train_acc 0.17734375596046448
batch_7 took 55.33659553527832s

train_acc 0.09375
batch_8 took 55.32691836357117s

train_acc 0.17499999701976776
batch_9 took 55.34224081039429s

iter 0 done, test_acc = 0.1120000034570694
iter 1 ======================= 
train_acc 0.11015625298023224
batch_0 took 55.32840847969055s



KeyboardInterrupt: 

## Matrix exp

In [ ]:
%%time

X_train, labels_train, X_test, labels_test = get_cifar_zca()

N = 1280

X_train      = torch.tensor(X_train[:N]).float()
X_test       = torch.tensor(X_test[:N]).float()
labels_train = torch.tensor(labels_train[:N], dtype=torch.long)
labels_test  = torch.tensor(labels_test[:N],  dtype=torch.long)

num_classes = 10

y_train = to_one_hot(labels_train, num_classes).to(device)
y_test  = to_one_hot(labels_test,  num_classes).to(device)

In [9]:
n_models = 5000

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

estimator = GpEstimator(models, num_classes, 1e4, X_train[0], device)

In [10]:
%%time
train_kernel, test_kernel = estimator.calc_kernels(X_train, X_test)

right_vector = calc_right_vector(train_kernel, y_train, learning_rate=1e5, reg_param=0)
y_pred = torch.matmul(test_kernel, right_vector).argmax(dim=1)

test_acc = (y_pred.cpu() == labels_test).float().mean()
print(test_acc)
print()

tensor(0.6102)

CPU times: user 1min 16s, sys: 25.5 s, total: 1min 42s
Wall time: 1min 42s


In [11]:
%%time
w_update = estimator.get_w_update(X_train, right_vector)

print(w_update.size())
print()

torch.Size([5000, 32, 10])

CPU times: user 34.7 s, sys: 14.7 s, total: 49.4 s
Wall time: 49.4 s


In [12]:
estimator.w = w_update

In [13]:
%%time

y_pred = estimator.predict(X_test).argmax(dim=1)

test_acc = (y_pred.cpu() == labels_test).float().mean()
print(test_acc)
print()

tensor(0.6102)

CPU times: user 35.2 s, sys: 14.9 s, total: 50.1 s
Wall time: 50 s
